In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import random

In [2]:
X = np.reshape([random()*100 for i in range(400)],(100,4))

In [3]:
col_max = np.amax(X,axis=0)
col_min = np.amin(X,axis=0)
centroid = (col_max + col_min)/2

In [4]:
Z = (X-centroid)**2

In [5]:
alpha = (1.0 / 100) * np.ones(100)

In [6]:
ZT = Z.T
h = np.matmul(alpha,Z)

In [43]:
# X[100,:]
index= np.random.randint(100)
index

0

In [45]:
alpha[99]

0.01

In [ ]:
def meepcalgo(n,):

  index= np.random.randint(n)
  # alpha_i=X[index,:]
  while(!converged):
    alpha_old = alpha[index]
    S=h
